### 3- Load and Infer From Huggingface Hub.

This notebook demonstrates the process of loading and evaluating two models for text classification using the AG News dataset. The two models used are:

1. **Pretrained Base Model with Probing**: A GPT-2 model fine-tuned for sequence classification.
2. **Pretrained Base Model with Low-Rank Adaptation (LoRA)**: A GPT-2 model fine-tuned using the LoRA technique for efficient adaptation.

Both models are loaded from Hugging Face's model hub and evaluated on a subset of the AG News dataset.

In [1]:
import gc
import torch

gc.collect()       # Python garbage collection
torch.cuda.empty_cache()  # Free up the GPU cache

In [2]:
# Import the datasets and transformers packages
from datasets import load_dataset

# Load the train and test splits of the AG News dataset
splits = ["train", "test"]  # Define the dataset splits to load
ds = {split: ds for split, ds in zip(splits, load_dataset("ag_news", split=splits))}  # Load the dataset and store it in a dictionary

# For each split, shuffle the dataset and select a subset of samples
for split in splits:
    print(f"{split}: {len(ds[split])//10} samples")  # Print the number of samples in the subset
    ds[split] = ds[split].shuffle(seed=42).select(range(len(ds[split]) // 10))  # Shuffle and select 10% of the dataset



train: 12000 samples
test: 760 samples


In [3]:
from transformers import AutoTokenizer
from peft import PeftModel, PeftConfig

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# # Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token



def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    return tokenizer(examples["text"], truncation=True, padding="max_length")


tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)


# Show the first example of the tokenized training set
print(tokenized_ds["train"][0]["input_ids"])

[43984, 75, 13410, 1582, 47557, 416, 8956, 29560, 7941, 423, 3181, 867, 11684, 290, 4736, 287, 19483, 284, 257, 17369, 11, 262, 1110, 706, 1248, 661, 3724, 287, 23171, 379, 257, 1964, 7903, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5

In [4]:
from peft import LoraConfig, PeftModelForSequenceClassification, TaskType, AutoPeftModelForSequenceClassification
# adapter_model="models/gpt2_ag_news_peft"
adapter_model="elewah/gpt2-ag-news-peft"
lora_model = AutoPeftModelForSequenceClassification.from_pretrained(adapter_model,
     num_labels=4,
    label2id={"World": 0, "Sports": 1, "Business": 2, "Sci/Tech": 3},
    id2label={0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"},
)
lora_model.config.pad_token_id = lora_model.config.eos_token_id

# Set the padding token ID for the model configuration
lora_model.config.pad_token_id = tokenizer.pad_token_id

print(lora_model)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B):

In [5]:
from transformers import AutoModelForSequenceClassification

base_model = AutoModelForSequenceClassification.from_pretrained("elewah/gpt2-ag-news")

base_model.config.pad_token_id = base_model.config.eos_token_id

# Set the padding token ID for the model configuration
base_model.config.pad_token_id = tokenizer.pad_token_id

print(base_model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


In [6]:
# Import necessary libraries
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

# Define a function to compute evaluation metrics
def compute_metrics(eval_pred):
    """
    Compute accuracy for the evaluation predictions.
    Args:
        eval_pred: A tuple containing predictions and labels.
    Returns:
        A dictionary with the accuracy metric.
    """
    predictions, labels = eval_pred
    # Get the index of the highest probability for each prediction
    predictions = np.argmax(predictions, axis=1)
    # Calculate accuracy by comparing predictions with labels
    return {"accuracy": (predictions == labels).mean()}



# Initialize the HuggingFace Trainer class
# The Trainer class simplifies the training and evaluation process for PyTorch models
lora_model_trainer = Trainer(
    model=lora_model,  # The model to be trained
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",  # Directory to save model checkpoints
        learning_rate=2e-3,  # Learning rate for the optimizer
        per_device_train_batch_size=1,  # Batch size for training
        per_device_eval_batch_size=1,  # Batch size for evaluation
        num_train_epochs=1,  # Number of training epochs
        weight_decay=0.01,  # Weight decay for regularization
        evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
        save_strategy="epoch",  # Save the model at the end of each epoch
        load_best_model_at_end=True,  # Load the best model at the end of training
    ),
    train_dataset=tokenized_ds["train"],  # Training dataset
    eval_dataset=tokenized_ds["test"],  # Evaluation dataset
    tokenizer=tokenizer,  # Tokenizer used for preprocessing
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # Data collator for padding sequences
    compute_metrics=compute_metrics,  # Function to compute evaluation metrics
)



# Initialize the HuggingFace Trainer class
# The Trainer class simplifies the training and evaluation process for PyTorch models
base_model_trainer = Trainer(
    model=base_model,  # The model to be trained
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",  # Directory to save model checkpoints
        learning_rate=2e-3,  # Learning rate for the optimizer
        per_device_train_batch_size=1,  # Batch size for training
        per_device_eval_batch_size=1,  # Batch size for evaluation
        num_train_epochs=1,  # Number of training epochs
        weight_decay=0.01,  # Weight decay for regularization
        evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
        save_strategy="epoch",  # Save the model at the end of each epoch
        load_best_model_at_end=True,  # Load the best model at the end of training
    ),
    train_dataset=tokenized_ds["train"],  # Training dataset
    eval_dataset=tokenized_ds["test"],  # Evaluation dataset
    tokenizer=tokenizer,  # Tokenizer used for preprocessing
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # Data collator for padding sequences
    compute_metrics=compute_metrics,  # Function to compute evaluation metrics
)



/opt/conda/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_175775/314283845.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  lora_model_trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/tmp/ipykernel_175775/314283845.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  base_model_trainer = Trainer(


In [7]:
original_performance = base_model_trainer.evaluate()
fine_tuned_performance = lora_model_trainer.evaluate()
print("Original Model:", original_performance)
print("Fine-Tuned Model:", fine_tuned_performance)

Original Model: {'eval_loss': 0.5844687819480896, 'eval_model_preparation_time': 0.0014, 'eval_accuracy': 0.8315789473684211, 'eval_runtime': 711.3713, 'eval_samples_per_second': 1.068, 'eval_steps_per_second': 1.068}
Fine-Tuned Model: {'eval_loss': 1.7015939950942993, 'eval_model_preparation_time': 0.0022, 'eval_accuracy': 0.7894736842105263, 'eval_runtime': 833.6598, 'eval_samples_per_second': 0.912, 'eval_steps_per_second': 0.912}
